# Depth-First-Search
# Connected Components
## Yunjun Xia

Implement depth-first search for identifying the connected components of an undirected graph and implement procedure Search as a subroutine along the way.

Dataset: recording the interactions between characters in Homer's *Iliad*.

In [1]:
import networkx
from urllib.request import urlopen
homer = urlopen('http://people.sc.fsu.edu/~jburkardt/datasets/sgb/homer.dat')
homer = [line.decode('utf-8') for line in homer]

The format of the data is straightforward. After some comment lines (beginning with \*), the file lists a codename for each character (i.e., node of the graph), followed by a description. The file then lists the groups of characters that interact in each chapter, from which we can form the edges. For instance, the first line has the form:

```1:CH,AG,ME,GS;AP,CH;HE,AC;AC,AG,CA;HE,AT;AT,AC;AT,OG;NE,AG,AC;CS,OD```

This means that CH,AG,ME,GS interacted, so there are edges for all pairs of these nodes. Groups of characters that interacted are separated by semicolons. The lines start with chapter information of the form `1:` or `&:`, which can be ignored for this problem.

First implement a function to read in the nodes from the input file. 

Next implement a function to read in the edges from the input file.

In [2]:
def read_nodes(gfile):
    """Reads in the nodes of the graph from the input file.
    
    Args:
        gfile: A handle for the file containing the graph data, starting at the top.
        
    Returns:
        A generator of the nodes in the graph, yielding a list of the form:
            ['CH', 'AG, 'ME', ...]
    """
    # TODO: implement function
    if gfile==None or len(gfile)==0: #check the input file is empty or not
        return('Error: Your input file is empty!')
    if type(gfile)!=list: #check the input file type is a list or not
        return('Error: Your input type is not a list')
    
    length=len(gfile) #get the number of strings in gfile
    nodes=[] #construct an empty string to store the nodes 
    for i in range(length):
        #use an if statement to get the lines of nodes
        if gfile[i][0]!='*' and gfile[i]!='\n' and ':' not in gfile[i]:
            templength=len(gfile[i])
            #use a for loop to find the first space in the specific line
            for j in range(templength):
                if gfile[i][j]==' ':
                    index=j
                    break
            tempnode=gfile[i][0:j] #get the node of the ith line
            nodes.append(tempnode) #add the node to the string
    return nodes
    pass


In [3]:
from itertools import combinations #import the combinations function from itertools for read_edges, which is permitted

def read_edges(gfile):
    """Reads in the edges of the graph from the input file.
    
    Args:
        gfile: A handle for the file containing the graph data, starting at the top 
            of the edges section.
            
    Returns:
        A generator of the edges in the graph, yielding a list of pairs of the form:
            [('CH', 'AG'), ('AG', 'ME'), ...]
    """
    # TODO: implement function
    if gfile==None or len(gfile)==0: #check the input file is empty or not
        return('Error: Your input file is empty!')
    if type(gfile)!=list: #check the input file type is a list or not
        return('Error: Your input type is not a list')
    
    length=len(gfile) #get the number of strings in gfile
    edges=[] #construct an empty string to store the edges
    for i in range(length):
        if ':' in gfile[i]: #get the lines of edge connections
            
            #get the temp string only including valid info. within the ith line
            #eg. if the ith line is '1:aa,bb;cc,dd\n', temp is 'aa,bb;cc,dd'
            start=gfile[i].find(':')+1
            end=len(gfile[i])-1
            temp=gfile[i][start:end]
            
            templist=[] #set an empty list
            count=0 #set count to 0
            
            #we divide the orginal temp string into separate subproblem, separated by ';'
            while temp.find(';')!=-1:
                index=temp.find(';')
                temp2=temp[0:index]
                for k in range(len(temp2)):
                    if temp2[k]==',':
                        templist.append(temp2[count:k])
                        count=k+1
                templist.append(temp2[count:len(temp2)])
                edges=edges+list(combinations(templist,2))
                count=0
                templist=[]
                temp=temp[index+1:len(temp)]
            
            #this part will be runned for two possibilities:
            #1. the ith line has no semicolons, so we skip the while loop to deal with the line entirely
            #2. the rest is for the last subproblem which is the part after the last semicolon in the ith line
            for p in range(len(temp)):
                if temp[p]==',':
                    templist.append(temp[count:p])
                    count=p+1
            templist.append(temp[count:len(temp)])
            edges=edges+list(combinations(templist,2))
    return edges
    pass


The following code should now correctly create the graph.

In [4]:
import networkx as nx
G = nx.Graph()
G.add_nodes_from(read_nodes(homer))
G.add_edges_from(read_edges(homer))

Next implement procedure Search. The function takes in a graph and a root node, and returns a list of the nodes visited during the search. The nodes should appear in the order in which they were *first visited*. The neighbors of a node should be processed in *alphabetical order*, where numbers come before letters. This will ensure that the output of the function is uniquely defined, given any input node.


In [5]:
def Search(graph, root):
    """Runs Search from vertex root in a graph. Neighboring nodes are processed in alphabetical order.
    
    Args:
        graph: the given graph, with nodes encoded as strings.
        root: the node from which to start the search.
        
    Returns:
        A list of nodes in the order in which they were first visited.
    """
    # TODO: implement function
    nodelist=[] #construct an empty string to store the nodes we visited in alphabetical order
    subsearch(graph,root,nodelist) #run subsearch to fully construct the nodelist
    
    #Important: here we set attribute 'explored' of all nodes back to 0 to assure that 
    #           the function Search can be runned repeatedly. However, this will not
    #           affect the function connected_components(graph), because in that function,
    #           the attribute we reset to 0 is 'exploredCC' instead of 'explored'. Also,
    #           we take consider of 'exploredCC' to check a node whether we have explored.
    for x in graph.nodes:
        graph.nodes[x]['explored']=0
    
    return nodelist
    pass

def subsearch(graph, root, nodelist):
    """This is an auxiliary function for Search. Runs subsearch from vertex root in a graph and append the 
       vertex root into nodelist. Mark attributed 'explored' and 'explored CC' to 1 for vertex root.
       
    Args:
        graph: the given graph, with nodes encoded as strings.
        root: the node from which to start the subsearch.
        nodelist: the list with explored nodes.
        
    Returns:
        None
    """
    #set both attributes 'explored' and 'exploredCC' to 1 for vertex root
    graph.nodes[root]['explored']=1 
    graph.nodes[root]['exploredCC']=1
    nodelist.append(root) #add root to nodelist
    for x in sorted(list(graph.neighbors(root))): #explore the neighbors of root in alphabetical order
        if 'explored' not in graph.nodes[x] or graph.nodes[x]['explored']!=1:
            subsearch(graph,x,nodelist) #recursive subsearch for neighbors of vertex root
    pass


We will check the correctness of your code by verifying that it correctly computes the DFS tree starting at Ulysses (node `OD`).

In [6]:
ulysses = Search(G, 'OD')

Next implement DFS to find the connected components of the character graph. When choosing roots for your components, always pick the *smallest unvisited node* according to alphabetical ordering. Combined with your Search routine, this will ensure that the output is again uniquely defined.

In [7]:
def connected_components(graph):
    """Computes the connected components of the given graph.
    
    Args: 
        graph: the given graph, with nodes encoded as strings.
        
    Returns:
        The connected components of the graph. Components are listed in
        alphabetical order of their root nodes.
    """
    # TODO: implement function
    nodes=sorted(list(graph.nodes)) #sort all the nodes in alphabetical order and store them in list nodes
    CC=[] #contruct an empty list to store output
    for x in nodes: #set the attribute 'exploredCC' of all the nodes to 0
        graph.nodes[x]['exploredCC']=0
    for y in nodes:
        if graph.nodes[y]['exploredCC']==0: #do Search for those nodes where we have not explored
            templist=Search(graph, y) #store the output of Search to templist
            CC.append(templist) #append the output Search to the output of Connected Components
    return CC
    pass


We will check correctness of your code by verifying that your output list is identical to our solution.

In [8]:
character_interactions = connected_components(G)

As a preliminary check, you should find that the following statements are all true.

In [9]:
component_sizes = [len(c) for c in character_interactions]
print("There are 12 connected components in the Iliad:{}".format(len(component_sizes) == 12))
print("The giant component has size 542:{}".format(max(component_sizes) == 542))
print("There are 5 isolated characters:{}".format(len([c for c in component_sizes if c == 1]) == 5))

There are 12 connected components in the Iliad:True
The giant component has size 542:True
There are 5 isolated characters:True
